In [1]:
import pandas as pd
import numpy as np

Считываем файлы

In [3]:
clust_adress = pd.read_csv('address_clust.csv')
stats_adress = pd.read_csv('address_stats.csv')

In [6]:
stats_adress.head(10)

,id,address_id,transaction_id,received,sent
0,87134765,46402336,19162323,0.0,1800000.0
1,87134766,45919235,19162323,0.0,1071052.0
2,87134767,46529090,19162323,1000000.0,0.0
3,87134768,46529091,19162323,1821052.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0
5,87154707,46539634,19166856,90000.0,0.0
6,87154708,46402336,19166856,900000.0,0.0
7,87187802,46553407,19174421,0.0,4800000.0
8,87187803,46557249,19174421,4590000.0,0.0
9,87187804,46529090,19174421,200000.0,0.0


In [5]:
clust_adress.head()

,address_id,cluster_id
0,71413451,1
1,71411914,1
2,71410369,1
3,71410100,1
4,71410040,1


Соединяем файлы, и заменяем NaN на 0

In [7]:
merged = stats_adress.merge(clust_adress, how='left', on='address_id')
merged['cluster_id']= merged['cluster_id'].apply(lambda x: 0 if np.isnan(x) else x )

In [10]:
merged.head()

,id,address_id,transaction_id,received,sent,cluster_id
0,87134765,46402336,19162323,0.0,1800000.0,0.0
1,87134766,45919235,19162323,0.0,1071052.0,0.0
2,87134767,46529090,19162323,1000000.0,0.0,1.0
3,87134768,46529091,19162323,1821052.0,0.0,0.0
4,87154706,46529090,19166856,0.0,1000000.0,1.0


In [13]:
merged.duplicated().value_counts() #проверка на дубликаты

False    54445
True     22279
dtype: int64

In [15]:
merged = merged.drop_duplicates() #удаляем дубликаты
len(merged)

54445

Проверка гипотезы о том, что в одной транзакции есть отправители только одного кластера.
Группируем по транзакции, аггрегируем количество отправлений и количество разных кластеров -> убеждаемся, что нет строк, 
где количество разных кластеров >1

In [20]:
sent = merged[merged['sent']>0]
groups_sent = sent.groupby(by = 'transaction_id').agg({'cluster_id':['count','nunique']})
groups_sent[groups_sent['cluster_id']['nunique']>1]

Empty DataFrame
Columns: [(cluster_id, count), (cluster_id, nunique)]
Index: []

Создаем функцию для суммирования отправленных биткоинов от кластера a кластеру b, 
вывод в биткоинах

In [24]:
def a_to_b_sum(a,b):
    #список транзакций, в которых кластер a был отправителем
    transactions_a = list(merged[(merged['cluster_id'] == a) & (merged['sent']>0)]['transaction_id'].unique()) 
    #сумма полученных сатош кластером b в транзакциях transactions_a
    a_to_b_sum = merged[(merged['transaction_id'].isin(transactions_a))&(merged['cluster_id'] == b)]['received'].sum() 
    return a_to_b_sum/10**8
    

Функция для подсчета комиссии, потраченной кластером a в биткоинах

In [25]:
def a_fee(a):
    #список транзакций, в которых кластер a был отправителем
    transactions_a = list(merged[(merged['cluster_id'] == a) & (merged['sent']>0)]['transaction_id'].unique())
    #сумма отправленных сатош кластером a в транзакциях transactions_a
    a_sent_sum = merged[(merged['transaction_id'].isin(transactions_a))]['sent'].sum()
    #сумма полученных сатош любым кластером в транзакциях transactions_a
    from_a_received_sum = merged[(merged['transaction_id'].isin(transactions_a))]['received'].sum()
    a_fee_sum = a_sent_sum - from_a_received_sum
    return a_fee_sum/10**8

Ответы на вопросы, полученные кластерами суммы за вычетом комиссий 

In [54]:
print('Кластер 1 отправил в кластер 2: ',a_to_b_sum(1,2), 'BTC')
print('Кластер 2 отправил в кластер 1: ',a_to_b_sum(2,1), 'BTC')
print('Кластер 1 отправил в кластер 0: ',a_to_b_sum(1,0), 'BTC')
print('Кластер 0 отправил в кластер 1: ',a_to_b_sum(0,1), 'BTC')
print('Кластер 2 отправил в кластер 0: ',a_to_b_sum(2,0), 'BTC')
print('Кластер 0 отправил в кластер 2: ',a_to_b_sum(0,2), 'BTC')
print('Кластер 1 потратил на fee: ',a_fee(1), 'BTC')
print('Кластер 2 потратил на fee: ',a_fee(2), 'BTC')

Кластер 1 отправил в кластер 2:  64.62162 BTC
Кластер 2 отправил в кластер 1:  67.91 BTC
Кластер 1 отправил в кластер 0:  291.5108346 BTC
Кластер 0 отправил в кластер 1:  288.7905886 BTC
Кластер 2 отправил в кластер 0:  2.43507 BTC
Кластер 0 отправил в кластер 2:  6.09640861 BTC
Кластер 1 потратил на fee:  0.502344 BTC
Кластер 2 потратил на fee:  0.37295861 BTC


In [55]:
#проверка сходимости сумм
merged['sent'].sum()/10**8 == round(a_to_b_sum(0,0)+a_to_b_sum(0,1)+a_to_b_sum(0,2)+a_to_b_sum(1,0)+a_to_b_sum(1,1)\
+a_to_b_sum(1,2)+a_to_b_sum(2,0)+a_to_b_sum(2,1)+a_to_b_sum(2,2)+a_fee(0)+a_fee(1)+a_fee(2),8)

True